# Lab 1

In [ ]:
# Code Block 1
scope.arm()
target.simpleserial_write("g", [])
scope.capture()

val = target.simpleserial_read('r', 4)
print(val)

In [ ]:
# Code Block 2
scope.arm()
target.simpleserial_write("g", [])
scope.capture()

In [ ]:
# Code Block 3
gc.set_range("width", 0, scope.glitch.phase_shift_steps // 2) # max range
gc.set_range("offset", 0, scope.glitch.phase_shift_steps) # max range

In [ ]:
# Code Block 4
scope.glitch.width = glitch_setting[0]
scope.glitch.offset = glitch_setting[1]
scope.glitch.ext_offset = glitch_setting[2]

In [ ]:
# Code Block 5
# NOTE: lots of ways to do this one, this is just one of the ways
gcnt = struct.unpack("<I", val['payload'])[0]

if gcnt != 2500:
    gc.add("success")
else:
    gc.add("normal")

# Lab 2

In [ ]:
# Code Block 1
pw = bytearray([0x00*5])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']

print(val)

In [ ]:
# Code Block 2
# Note: These are just example ranges and may not work for your device
gc.set_range("width", 1900, 2100)
gc.set_range("offset", 2400, 3000)
gc.set_range("ext_offset", 0, 150)
gc.set_global_step([20])

In [ ]:
# Code Block 3
val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10, timeout=50)#For loop check
if val['valid'] is False:
    gc.add("reset")
else:
    if val['payload'] == bytearray([1]): #for loop check
        successes +=1 
        gc.add("success")
        print(val)
        print(val['payload'])
        print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
        print("🐙", end="")
    else:
        gc.add("normal")

# Lab 3

In [ ]:
# Code Block 1
sig = output['payload']
sig_check = signer.verify(hash_object, sig)
if sig_check is False: 
    # detect if the calculation was messed up
    # call the faulty signature whatever you want
    # but we'll assume it's called sig for the rest of the lab
    print("Glitched!")
    sig = sig
    break

# Lab 4

In [ ]:
# Code Block 1
response, responsehex = tx_rx(b"\x20", 4, 4)
responsehex

# Lab 5

In [ ]:
# Code Block 1
import struct

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)
scope.adc.timeout = 0.5 # prevent crashes from taking too long

reboot_flush()

corrupt_ext_offset = 0

# 1. Iterate through the loop
for glitch_settings in gc.glitch_values():
    scope.glitch.ext_offset = glitch_settings[2]
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]

    target.flush() # flush garbage from serial lines
    
    # Try detecting if the target is crashed
    if scope.adc.state:
        gc.add("reset")
        reboot_flush()

    # 3. Arm the scope
    scope.arm()

    # 4. Send the "g" command
    target.simpleserial_write("g", bytearray([]))

    # Call scope.capture
    ret = scope.capture()
    
    # If trigger didn't fire, target is probably crashed
    if ret:
        gc.add("reset")
        reboot_flush()
    else:
        # 6. Read back the calculated number
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=1, timeout=50)
        
        # If target sent us garabge, it probably crashed
        if val['valid'] is False:
            gc.add("reset")
            reboot_flush()
        else:
            if val['rv'][0] == 0x10:
                print("Glitched loop at {}".format(scope.glitch.ext_offset))
                print("settings = {}".format(glitch_settings))
                gc.add("partial_success")
                corrupt_ext_offset = scope.glitch.ext_offset
                print(val['payload'])
                break
            
print("Done glitching")

# reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

In [ ]:
# Code Block 2
import struct

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)
scope.adc.timeout = 0.5 # prevent crashes from taking too long

reboot_flush()

# 1. Iterate through the loop
for glitch_settings in gc.glitch_values():
    scope.glitch.ext_offset = [scope.glitch.ext_offset[0], glitch_settings[2]]
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]

    target.flush() # flush garbage from serial lines

    # Try detecting if the target is crashed
    if scope.adc.state:
        gc.add("reset")
        reboot_flush()

    # 3. Arm the scope
    scope.arm()

    # 4. Send the "g" command
    target.simpleserial_write("g", bytearray([]))

    # Call scope.capture
    ret = scope.capture()
    
    # If trigger didn't fire, target is probably crashed
    if ret:
        gc.add("reset")
        reboot_flush()
    else:
        # 6. Read back the calculated number
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=1, timeout=50)
        
        # If target sent us garabge, it probably crashed
        if val['valid'] is False:
            gc.add("reset")
            reboot_flush()
        else:
            gcnt = struct.unpack("<I", val['payload'])[0]
            if gcnt == 0:
                gc.add("partial_success")
            elif gcnt == 2500:
                #print("No effect")
                gc.add("normal")
            else:
                print("Successful glitch! gcnt = {}".format(gcnt))
                print("settings = {}".format(glitch_settings))
                gc.add("success")
                break
            
print("Done glitching")

# reenable logging
cw.set_all_log_levels(cw.logging.WARNING)